# Trials Generation Script

## Loads all stimuli details from a CSV and generates trial sets

### Key features of each stimulus are also included in dataframe rows (CSV rows). 
 
Raúl Arrabales - Change Blindness in Hierarchical Stimuli Project

In [198]:
from os import listdir
from os.path import isfile, join

import pandas as pd

In [199]:
# Load dataframe with all stimuli
sdf = pd.read_csv('AllStimuli.csv',sep=';')

In [200]:
# Stimuli table
sdf.head(10)

,StimulusId,FileName,Pattern,G1,L1,G2,L2,G3,L3,G4,L4
0,0,1Row-Xf-Xd-Xd-Xf,1Row,X,f,X,d,X,d,X,f
1,1,1Row-De-Bg-Zf-Hd,1Row,D,e,B,g,Z,f,H,d
2,2,1Row-Xd-Xg-Xd-Xg,1Row,X,d,X,g,X,d,X,g
3,3,1Row-Hx-Od-Xf-Ah,1Row,H,x,O,d,X,f,A,h
4,4,1Row-Xd-Xf-Xa-Xg,1Row,X,d,X,f,X,a,X,g
5,5,1Row-Dx-Bg-Ef-Hd,1Row,D,x,B,g,E,f,H,d
6,6,1Row-Oa-Rf-Xd-Ae,1Row,O,a,R,f,X,d,A,e
7,7,1Row-Ae-Xd-Zx-Hd,1Row,A,e,X,d,Z,x,H,d
8,8,1Row-Zg-Bg-Ef-Hd,1Row,Z,g,B,g,E,f,H,d
9,9,1Row-Og-Bh-Xg-Bf,1Row,O,g,B,h,X,g,B,f


In [201]:
# Stimuli table stats
sdf.describe()

,StimulusId
count,98.000000
mean,48.500000
std,28.434134
min,0.000000
25%,24.250000
50%,48.500000
75%,72.750000
max,97.000000


In [202]:
# Dataframe to store all trials
trial_columns = [
    'TrialId',     # Trial ID
    'S1',          # Stimulus 1
    'S2',          # Stimulus 2
    'S1I',         # S1I (S1 interval) in ms
    'ISI',         # ISI (inter-stimulus interval) is ms
    'S2I',         # S2I (S2 interval) in ms
    'ChangeType',  # Type of change (L:local, G:global, M:local and global, N:no change, Z: Multiple changes)
    'Correct',     # Correct key to press (L, G, N, M, Z)
    'ChangePos',   # Position of the change [1..4] (0 for no change)
    'Lbefore',     # Local letter before change
    'Lafter',      # Local letter after change
    'Gbefore',     # Global letter before change
    'Gafter']      # Global letter after change
trials_df = pd.DataFrame(columns=trial_columns)
trials_df.head()

,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter


In [203]:
# Number of local changes between two stimuli
def LocalChanges( S1, S2 ):
    numLocalChanges = 0
    
    if S1['L1'] != S2['L1']:
        numLocalChanges +=1
    if S1['L2'] != S2['L2']:
        numLocalChanges +=1
    if S1['L3'] != S2['L3']:
        numLocalChanges +=1
    if S1['L4'] != S2['L4']:
        numLocalChanges +=1    
        
    return numLocalChanges


In [204]:
# Number of global changes between two stimuli
def GlobalChanges( S1, S2 ):
    numGlobalChanges = 0
    
    if S1['G1'] != S2['G1']:
        numGlobalChanges +=1
    if S1['G2'] != S2['G2']:
        numGlobalChanges +=1
    if S1['G3'] != S2['G3']:
        numGlobalChanges +=1
    if S1['G4'] != S2['G4']:
        numGlobalChanges +=1 
        
    return numGlobalChanges


In [205]:
# Given one local and one global change, True if both changes are in the same letter (position)
def MChange( S1, S2 ):
    posLocal = 0
    posGlobal = 0
    
    if S1['L1'] != S2['L1']:
        posLocal = 1
    elif S1['L2'] != S2['L2']:
        posLocal = 2
    elif S1['L3'] != S2['L3']:
        posLocal = 3
    elif S1['L4'] != S2['L4']:
        posLocal = 4
        
    if S1['G1'] != S2['G1']:
        posGlobal = 1
    if S1['G2'] != S2['G2']:
        posGlobal = 2
    if S1['G3'] != S2['G3']:
        posGlobal = 3
    if S1['G4'] != S2['G4']:
        posGlobal = 4
        
    return posLocal == posGlobal
    

In [206]:
# Given one local change, returns the position of that change [1..4]
def LocalChangePos( S1, S2 ):
    posLocal = 0
        
    if S1['L1'] != S2['L1']:
        posLocal = 1
    elif S1['L2'] != S2['L2']:
        posLocal = 2
    elif S1['L3'] != S2['L3']:
        posLocal = 3
    elif S1['L4'] != S2['L4']:
        posLocal = 4
        
    return posLocal


In [207]:
# Given one global change, returns the position of that change [1..4]
def GlobalChangePos( S1, S2 ):
    posGlobal = 0
    
    if S1['G1'] != S2['G1']:
        posGlobal = 1
    if S1['G2'] != S2['G2']:
        posGlobal = 2
    if S1['G3'] != S2['G3']:
        posGlobal = 3
    if S1['G4'] != S2['G4']:
        posGlobal = 4
        
    return posGlobal


In [208]:
# Gets the letter position whenever only one change ocurred between S1 and S2
def ChangePos( type, S1, S2 ):
    if type == 'L':
        return LocalChangePos( S1, S2 )
    elif type == 'G':
        return GlobalChangePos( S1, S2 )
    else:
        return 0


In [209]:
# Iterate over all rows combined with all (same) rows - NxN combinations

trials_df = pd.DataFrame(columns=trial_columns) # Reset trials df
i = 0      # trial id
ISI = 250  # default inter-stimulus interval
S1I = 500  # default S1 interval
S2I = 500  # default S2 interval

print('Processing...')

for index1, row1 in sdf.iterrows():
    for index2, row2 in sdf.iterrows():

        # print('Processing trial Id '+str(i)+': S'+str(index1)+' and S'+str(index2) )
        i += 1
        
        # N - No Change from S1 to S2
        if index1 == index2: 
            change = 'N' # Same stimulus means no change
            
        # L - One Local Change from S1 to S2
        elif LocalChanges(row1, row2) == 1 and GlobalChanges(row1, row2) == 0:
            change = 'L'
        
        # G - One Global Change from S1 to S2
        elif LocalChanges(row1, row2) == 0 and GlobalChanges(row1, row2) == 1:
            change = 'G'
                        
        # M - One Local and One Global Change from S1 to S2
        elif LocalChanges(row1, row2) == 1 and GlobalChanges(row1, row2) == 1 and MChange(row1, row2):
            change = 'M'
            
        # Z - More than one change in same letter
        else:
            change = 'Z'

        # Get the position of the change
        posChange = ChangePos(change, row1, row2)
        
        # Get the letters changed
        Lbefore = '*'
        Lafter = '*'
        Gbefore = '*'
        Gafter = '*'
        
        if change == 'L':
            letterindex = 'L' + str(posChange)
            Lbefore = row1[letterindex].upper()
            Lafter = row2[letterindex].upper()
        elif change == 'G':
            letterindex = 'G' + str(posChange)
            Gbefore = row1[letterindex].upper()
            Gafter = row2[letterindex].upper()
            
        # Add a row to trials dataframe
        trials_df.loc[len(trials_df)] = [
            str(i),                # Trial ID
            row1['FileName'],      # File name of S1
            row2['FileName'],      # File name of S2
            str(S1I),              # S1 display interval
            str(ISI),              # Inter stimulus interval
            str(S2I),              # S2 display interval
            change,                # Type of change
            change,                # Correct answer letter
            str(posChange),        # Position of change
            Lbefore,               # Local letter before change
            Lafter,                # Local letter after change
            Gbefore,               # Global letter before change
            Gafter]                # Glboal letter after change
        
print('Done. Processed: ' +str(i)+ ' trials.')
        
                   

Processing...
Done. Processed: 9604 trials.


In [210]:
print('Total number of trials generated: ' + str(len(trials_df)))

Total number of trials generated: 9604


In [211]:
# Trials Dataframe samlple: 
trials_df.head(10)

,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
0,1,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xf,500,250,500,N,N,0,*,*,*,*
1,2,1Row-Xf-Xd-Xd-Xf,1Row-De-Bg-Zf-Hd,500,250,500,Z,Z,0,*,*,*,*
2,3,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xg-Xd-Xg,500,250,500,Z,Z,0,*,*,*,*
3,4,1Row-Xf-Xd-Xd-Xf,1Row-Hx-Od-Xf-Ah,500,250,500,Z,Z,0,*,*,*,*
4,5,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xf-Xa-Xg,500,250,500,Z,Z,0,*,*,*,*
5,6,1Row-Xf-Xd-Xd-Xf,1Row-Dx-Bg-Ef-Hd,500,250,500,Z,Z,0,*,*,*,*
6,7,1Row-Xf-Xd-Xd-Xf,1Row-Oa-Rf-Xd-Ae,500,250,500,Z,Z,0,*,*,*,*
7,8,1Row-Xf-Xd-Xd-Xf,1Row-Ae-Xd-Zx-Hd,500,250,500,Z,Z,0,*,*,*,*
8,9,1Row-Xf-Xd-Xd-Xf,1Row-Zg-Bg-Ef-Hd,500,250,500,Z,Z,0,*,*,*,*
9,10,1Row-Xf-Xd-Xd-Xf,1Row-Og-Bh-Xg-Bf,500,250,500,Z,Z,0,*,*,*,*


In [212]:
# Trials with no change from S1 to S2
N_trials = trials_df[trials_df['ChangeType'] == 'N']
print(str(len(N_trials)) + ' trials with no change:')
N_trials.head()

98 trials with no change:


,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
0,1,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xf,500,250,500,N,N,0,*,*,*,*
99,100,1Row-De-Bg-Zf-Hd,1Row-De-Bg-Zf-Hd,500,250,500,N,N,0,*,*,*,*
198,199,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xg-Xd-Xg,500,250,500,N,N,0,*,*,*,*
297,298,1Row-Hx-Od-Xf-Ah,1Row-Hx-Od-Xf-Ah,500,250,500,N,N,0,*,*,*,*
396,397,1Row-Xd-Xf-Xa-Xg,1Row-Xd-Xf-Xa-Xg,500,250,500,N,N,0,*,*,*,*


In [213]:
# Trials with one local change from S1 to S2
L_trials = trials_df[trials_df['ChangeType'] == 'L']
print(str(len(L_trials)) + ' trials with one local change (L):')
L_trials.head()

138 trials with one local change (L):


,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
37,38,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xg,500,250,500,L,L,4,F,G,*,*
208,209,1Row-Xd-Xg-Xd-Xg,1Row-Xf-Xg-Xd-Xg,500,250,500,L,L,1,D,F,*,*
217,218,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xg-Xg-Xg,500,250,500,L,L,3,D,G,*,*
241,242,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xf-Xd-Xg,500,250,500,L,L,2,G,F,*,*
284,285,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xd-Xd-Xg,500,250,500,L,L,2,G,D,*,*


In [214]:
# Trials with one global change from S1 to S2
G_trials = trials_df[trials_df['ChangeType'] == 'G']
print(str(len(G_trials)) + ' trials with one global change (G):')
G_trials.head()

94 trials with one global change (G):


,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
133,134,1Row-De-Bg-Zf-Hd,1Row-De-Bg-Ef-Hd,500,250,500,G,G,3,*,*,Z,E
362,363,1Row-Hx-Od-Xf-Ah,1Row-Hx-Rd-Xf-Ah,500,250,500,G,G,2,*,*,O,R
368,369,1Row-Hx-Od-Xf-Ah,1Row-Zx-Od-Xf-Ah,500,250,500,G,G,1,*,*,H,Z
516,517,1Row-Dx-Bg-Ef-Hd,1Row-Zx-Bg-Ef-Hd,500,250,500,G,G,1,*,*,D,Z
526,527,1Row-Dx-Bg-Ef-Hd,1Row-Dx-Zg-Ef-Hd,500,250,500,G,G,2,*,*,B,Z


In [196]:
# Trials with one global and one local change from S1 to S2
M_trials = trials_df[trials_df['ChangeType'] == 'M']
print(str(len(M_trials)) + ' trials with one global and local change (M):')
M_trials.head()

46 trials with one global and local change (M):


,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
195,196,1Row-Dx-Bg-Ef-Hd,1Row-Gh-Bg-Ef-Hd,500,250,500,M,M,0,*,*,*,*
232,233,1Row-Dx-Bg-Ef-Hd,1Row-Ef-Bg-Ef-Hd,500,250,500,M,M,0,*,*,*,*
283,284,1Row-Gl-Bg-Ah-Hd,1Row-Ae-Bg-Ah-Hd,500,250,500,M,M,0,*,*,*,*
363,364,1Row-Hc-Rd-Xf-Ah,1Row-Hc-Bh-Xf-Ah,500,250,500,M,M,0,*,*,*,*
537,538,1Row-Hc-Bh-Xf-Ah,1Row-Hc-Rd-Xf-Ah,500,250,500,M,M,0,*,*,*,*


In [197]:
# Trials with multiple changes from S1 to S2
Z_trials = trials_df[trials_df['ChangeType'] == 'Z']
print(str(len(Z_trials)) + ' trials with multiple changes (Z):')
Z_trials.head()

3244 trials with multiple changes (Z):


,TrialId,S1,S2,S1I,ISI,S2I,ChangeType,Correct,ChangePos,Lbefore,Lafter,Gbefore,Gafter
1,2,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xg-Xd-Xg,500,250,500,Z,Z,0,*,*,*,*
2,3,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xf-Xa-Xg,500,250,500,Z,Z,0,*,*,*,*
3,4,1Row-Xf-Xd-Xd-Xf,1Row-Dx-Bg-Ef-Hd,500,250,500,Z,Z,0,*,*,*,*
4,5,1Row-Xf-Xd-Xd-Xf,1Row-Gl-Bg-Ah-Hd,500,250,500,Z,Z,0,*,*,*,*
5,6,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xg-Xd-Xg,500,250,500,Z,Z,0,*,*,*,*


In [175]:
# Export trials dataframe to a CSV file:
trials_df.to_csv('AllTrials.csv', sep =';', index_label='Id')